In [1]:
import pickle
import numpy as np
import pandas as pd
import re

seed = 2023

In [2]:
import os
os.chdir("../../")

In [3]:
from nlpsig_networks.scripts.ffn_baseline_functions import (
    ffn_hyperparameter_search,
)

/storage/ttseriotou/nlpsig-networks-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Rumours

In [4]:
%run notebooks/Rumours/load_sbert-embeddings.py

In [5]:
df_rumours.head()

,id,label,datetime,text,timeline_id
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0


In [6]:
sbert_embeddings.shape

torch.Size([5568, 384])

Baseline: FFN

In [9]:
num_epochs = 100
hidden_dim_sizes = [[32,32]]#[[32,32], [64,64], [128,128], [256,256]]
dropout_rates = [0.2] #[0.5, 0.2, 0.1]
learning_rates = [1e-4]#[1e-3, 1e-4, 5e-4]
seeds = [0, 1, 12, 123, 1234]
loss = "focal"
gamma = 2
validation_metric = "f1"

We use the ffn_hyperparameter_search function which loops through the different hidden dimensions, dropout rates and learning rates to find the best model for the validation set. We evaluate the model on several seeds and average the performance over the seeds.

In [11]:
ffn_current, best_ffn_current, _, __ = ffn_hyperparameter_search( 
    num_epochs=num_epochs,
    x_data=sbert_embeddings,
    y_data=y_data,
    hidden_dim_sizes=hidden_dim_sizes,
    output_dim=output_dim,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    data_split_seed=123,
    split_ids= torch.tensor(df_rumours['timeline_id'].astype(int)),
    k_fold=False,
    validation_metric=validation_metric,
    results_output=None,
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]


[INFO] Splitting data by provided groups
[INFO] Splitting data by provided groups
[INFO] Splitting data by provided groups
[INFO] Splitting data by provided groups
[INFO] Splitting data by provided groups



100%|██████████| 1/1 [00:27<00:00, 27.05s/it]


[INFO] Splitting data by provided groups
[INFO] Splitting data by provided groups
[INFO] Splitting data by provided groups
[INFO] Splitting data by provided groups
[INFO] Splitting data by provided groups


In [12]:
ffn_current

,loss,accuracy,f1,f1_scores,valid_loss,valid_accuracy,valid_f1,valid_f1_scores,hidden_dim,dropout_rate,learning_rate,seed,gamma,k_fold,model_id
0,focal,0.5784,0.514008,"[0.6909090909090909, 0.3371069182389937]",0.237245,0.639383,0.625018,"[0.6984126984126985, 0.551622418879056]","[32, 32]",0.2,0.0001,0,2,False,0
0,focal,0.5760,0.511569,"[0.6889671361502347, 0.3341708542713568]",0.231630,0.637011,0.624646,"[0.6927710843373495, 0.5565217391304348]","[32, 32]",0.2,0.0001,1,2,False,0
0,focal,0.5840,0.525166,"[0.6923076923076923, 0.35802469135802467]",0.237621,0.640569,0.626431,"[0.6991062562065542, 0.5537555228276877]","[32, 32]",0.2,0.0001,12,2,False,0
0,focal,0.5736,0.513608,"[0.6844286560094731, 0.342786683107275]",0.241144,0.638197,0.625025,"[0.6953046953046952, 0.5547445255474452]","[32, 32]",0.2,0.0001,123,2,False,0
0,focal,0.5736,0.514229,"[0.6840545346769413, 0.34440344403444034]",0.234256,0.635824,0.623917,"[0.6908358509566969, 0.556998556998557]","[32, 32]",0.2,0.0001,1234,2,False,0


In [14]:
best_ffn_current

,loss,accuracy,f1,f1_scores,valid_loss,valid_accuracy,valid_f1,valid_f1_scores,hidden_dim,dropout_rate,learning_rate,seed,gamma,k_fold
0,focal,0.5784,0.514008,"[0.6909090909090909, 0.3371069182389937]",0.237245,0.639383,0.625018,"[0.6984126984126985, 0.551622418879056]","[32, 32]",0.2,0.0001,0,2,False
0,focal,0.5760,0.511569,"[0.6889671361502347, 0.3341708542713568]",0.231630,0.637011,0.624646,"[0.6927710843373495, 0.5565217391304348]","[32, 32]",0.2,0.0001,1,2,False
0,focal,0.5840,0.525166,"[0.6923076923076923, 0.35802469135802467]",0.237621,0.640569,0.626431,"[0.6991062562065542, 0.5537555228276877]","[32, 32]",0.2,0.0001,12,2,False
0,focal,0.5736,0.513608,"[0.6844286560094731, 0.342786683107275]",0.241144,0.638197,0.625025,"[0.6953046953046952, 0.5547445255474452]","[32, 32]",0.2,0.0001,123,2,False
0,focal,0.5736,0.514229,"[0.6840545346769413, 0.34440344403444034]",0.234256,0.635824,0.623917,"[0.6908358509566969, 0.556998556998557]","[32, 32]",0.2,0.0001,1234,2,False


In [15]:
best_ffn_current["f1"].mean()

0.5157159701063523

In [16]:
np.stack(best_ffn_current["f1_scores"]).mean(axis=0)

array([0.68813342, 0.34329852])